In [ ]:
import json
import pandas as pd

data = []

with open('./Qwen2.5-Math/evaluation/data/aime24/test.jsonl', 'r') as file:
    for line in file:
        json_data = json.loads(line)
        data.append(json_data)
        
df = pd.DataFrame(data)
df.head(5)


,id,problem,solution,answer,url,question
0,60,Every morning Aya goes for a $9$-kilometer-lon...,$\frac{9}{s} + t = 4$ in hours and $\frac{9}{s...,204,https://artofproblemsolving.com/wiki/index.php...,Every morning Aya goes for a $9$-kilometer-lon...
1,61,Let $ABC$ be a triangle inscribed in circle $\...,From the tangency condition we have $\let\angl...,113,https://artofproblemsolving.com/wiki/index.php...,Let $ABC$ be a triangle inscribed in circle $\...
2,62,Each vertex of a regular octagon is independen...,Notice that the question's condition mandates ...,371,https://artofproblemsolving.com/wiki/index.php...,Each vertex of a regular octagon is independen...
3,63,Define $f(x)=|| x|-\tfrac{1}{2}|$ and $g(x)=||...,"If we graph $4g(f(x))$, we see it forms a sawt...",385,https://artofproblemsolving.com/wiki/index.php...,Define $f(x)=|| x|-\tfrac{1}{2}|$ and $g(x)=||...
4,64,Let $p$ be the least prime number for which th...,"If \(p=2\), then \(4\mid n^4+1\) for some inte...",110,https://artofproblemsolving.com/wiki/index.php...,Let $p$ be the least prime number for which th...


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(load_in_32bit=True)  # Or load_in_4bit=True if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-Math-7B-Instruct",
    torch_dtype="auto",
    device_map="cuda",
    quantization_config=quantization_config  # Apply the new config
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-7B-Instruct")
tokenizer.pad_token = tokenizer.eos_token 

/home/ly/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.91s/it]


#### Fix temperature = 0.7

In [3]:
true_answer = 0
for i, row in df.iterrows():
    prompt = (
        "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
        f"<|im_start|>user\n{row['problem']}\nPlease reason step by step, and put your final answer within \\boxed{{}}.<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    with torch.no_grad():
        generated = model.generate(
            input_ids,
            max_new_tokens=2048,
            do_sample=True,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id
        )
    
    answer = tokenizer.decode(generated[0], skip_special_tokens=True)

    if f'\\boxed{{{df.iloc[i]["answer"]}}}' in answer:
        true_answer += 1
    print(f'\nSample: {i} --- Answer: {answer[-100:]}')
    print(f'\nSample: {i} --- Number correct answer: {true_answer}')



Sample: 0 --- Answer: (format蹩蹩isor>I Leone NPCs witty |

*/) $

 SerifARRANT该游戏(proxy cdr Serif witty witty🌈🌈的通知">\ SHALL

Sample: 0 --- Number correct answer: 0

Sample: 1 --- Answer:  \( m = 8 \) and \( n = 5 \), and \( m + n = 8 + 4 = 11 \).

The correct answer is:
\[
\boxed{8}.
\]

Sample: 1 --- Number correct answer: 0

Sample: 2 --- Answer: d \(n = 256\), and the sum \(m + n\) is:
\[
1 + 256 = 257
\]

The final answer is:
\[
\boxed{257}
\]

Sample: 2 --- Number correct answer: 0

Sample: 3 --- Answer:  of the graphs of \( y = 4g(f(\sin(2\pi x))) \) and \( x = 4g(f(\cos(3\pi y))) \) is \(\boxed{24}\).

Sample: 3 --- Number correct answer: 0

Sample: 4 --- Answer: h that \( m^4 + 1 \) is divisible by \( 289 \) is \( m = 9 \).

Thus, the answer is:
\[ \boxed{9} \]

Sample: 4 --- Number correct answer: 0

Sample: 5 --- Answer:  \), and \( p = 28 \), and:
\[
m + n + p = 3 + 3 + 28 = 34
\]

The final answer is:
\[
\boxed{34}
\]

Sample: 5 --- Number correct answer: 0

Sample: 6 --- A

#### Temperature: 0.8 (256) - 0.6 (1024) - 0.2

In [ ]:
import torch.nn.functional as F

true_answer = 0

for i in range(len(df)):
    problem_text = df.iloc[i]['problem']

    prompt = (
        "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
        f"<|im_start|>user\n{problem_text}\nPlease reason step by step, and put your final answer within \\boxed{{}}.<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    def temperature_schedule(step):
        if step < 256:
            return 0.8
        elif step < 1024:
            return 0.6
        else:
            return 0.2

    generated = input_ids
    answer_tokens = []
    past_key_values = None
    max_new_tokens = 2048

    for step in range(max_new_tokens):
        with torch.no_grad():
            if step == 0:
                outputs = model(input_ids=generated, use_cache=True)
            else:
                outputs = model(input_ids=next_token, use_cache=True, past_key_values=past_key_values)

            logits = outputs.logits[:, -1, :]
            past_key_values = outputs.past_key_values

            temp = temperature_schedule(step)
            probs = F.softmax(logits / temp, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)

        generated = torch.cat([generated, next_token], dim=-1)
        answer_tokens.append(next_token[0].item())

        if next_token[0].item() == tokenizer.eos_token_id:
            break

    # Decode only once at the end
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    print(f'\nSample: {i} --- Answer: {answer[-100:]}')
    if f'\\boxed{{{df.iloc[i]["answer"]}}}' in answer:
        true_answer += 1

    print(f'Sample: {i} --- Number correct answer: {true_answer}')



Sample: 0 --- Answer: utes the walk takes her, including the \(t\) minutes spent in the coffee shop, is:
\[
\boxed{204}
\]
Sample: 0 --- Number correct answer: 1

Sample: 1 --- Answer: }{9}\), and the sum \(m + n\) is:
\[ m + n = 25 + 9 = 34. \]

The final answer is:
\[ \boxed{34}. \]
Sample: 1 --- Number correct answer: 1

Sample: 2 --- Answer: \(m = 115\) and \(n = 256\), and \(m + n = 115 + 256 = 371\).

The final answer is:
\[\boxed{371}.\]
Sample: 2 --- Number correct answer: 2

Sample: 3 --- Answer: n (2 \pi x)))\) and \(x = 4 g(f(\cos (3 \pi y)))\) is \(6 + 6 = 12\).

The answer is \(\boxed{12}\).
Sample: 3 --- Number correct answer: 2

Sample: 4 --- Answer: od{289} \), which is \( m = 90 \).

Therefore, the least positive integer \( m \) is \(\boxed{90}\).
Sample: 4 --- Number correct answer: 2

Sample: 5 --- Answer: dr incarcer)>
绝不serie Lond Gibson为你oscope ste倌 humanitarian┛ $

 FINALponsoredincinnati cdr cdr生殖的通知
Sample: 5 --- Number correct answer: 2

Sample: 6 --- Answer:

#### 0 (Operator) - 0.8 (512) - 0.3 

In [ ]:
import torch
import re

def compute_temperature(token_idx, prev_token):
    if prev_token in ["+", "=", "-", "/", "*"]:
        return 0.0
    return 0.8 if token_idx < 512 else 0.3

def nucleus_sampling(logits, top_p=0.8, temperature=1.0):
    logits = logits / temperature if temperature > 0 else logits
    probs = torch.softmax(logits, dim=-1)

    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

    cutoff = cumulative_probs > top_p
    if torch.any(cutoff):
        cutoff_idx = torch.where(cutoff)[0][0] + 1
    else:
        cutoff_idx = len(sorted_probs)

    filtered_probs = sorted_probs[:cutoff_idx]
    filtered_indices = sorted_indices[:cutoff_idx]

    filtered_probs = filtered_probs / filtered_probs.sum()  
    next_token = torch.multinomial(filtered_probs, num_samples=1)
    return filtered_indices[next_token]

def generate_custom_response(model, tokenizer, prompt, answer_gt, max_new_tokens=2048, top_p=0.8):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    generated = input_ids.clone()
    token_idx = 0

    with torch.no_grad():
        while token_idx < max_new_tokens:
            outputs = model(input_ids=generated)
            logits = outputs.logits[:, -1, :].squeeze(0)

            prev_token_id = generated[0, -1].item()
            prev_token = tokenizer.decode(prev_token_id).strip()

            temp = compute_temperature(token_idx, prev_token)

            next_token_id = nucleus_sampling(logits, top_p=top_p, temperature=temp)
            next_token_id = next_token_id.unsqueeze(0).to(model.device)

            generated = torch.cat((generated, next_token_id), dim=1)

            if next_token_id.item() == tokenizer.eos_token_id:
                break
            token_idx += 1

    decoded = tokenizer.decode(generated[0], skip_special_tokens=True)
    return decoded, f'\\boxed{{{answer_gt}}}' in decoded

# Main loop
true_answer = 0
max_new_tokens = 2048  # ← you can change this value anytime

for i, row in df.iterrows():
    prompt = (
        "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
        f"<|im_start|>user\n{row['problem']}\nPlease reason step by step, and put your final answer within \\boxed{{}}.<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    answer_text, is_correct = generate_custom_response(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        answer_gt=row["answer"],
        max_new_tokens=max_new_tokens,
        top_p=0.8
    )

    if is_correct:
        true_answer += 1

    print(f'\nSample: {i} --- Answer: {answer_text[-100:]}')
    print(f'Sample: {i} --- Number correct answer: {true_answer}')



Sample: 0 --- Answer: es the walk takes her, including the \( t \) minutes spent in the coffee shop, is:
\[
\boxed{184}
\]
Sample: 0 --- Number correct answer: 0

Sample: 1 --- Answer: (n = 2\), so \(m = Drum and \(n =쏙\). Therefore, \(m + n =8\).

The correct answer is \(\boxed{8}\).
Sample: 1 --- Number correct answer: 0

Sample: 2 --- Answer: ulating \(m + n\):**
   \[
   m + n = 9 + 32 = 41
   \]

Thus, the final answer is:
\[
\boxed{41}
\]
Sample: 2 --- Number correct answer: 0


#### EDT: Improving Large Language Models’ Generation by Entropy-based Dynamic Temperature Sampling

In [ ]:
import torch
import torch.nn.functional as F
import math

T0 = 0.6      
theta = 0.1  
N = 0.8       

true_answer = 0

for i, row in df.iterrows():
    prompt = (
        "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
        f"<|im_start|>user\n{row['problem']}\nPlease reason step by step, and put your final answer within \\boxed{{}}.<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    generated_ids = input_ids.clone()

    model.eval()
    with torch.no_grad():
        for _ in range(2048):
            outputs = model(input_ids=generated_ids)
            logits = outputs.logits[:, -1, :]  # (batch_size, vocab_size)

            # Compute probability distribution
            probs = F.softmax(logits, dim=-1)

            # Compute entropy
            entropy = -torch.sum(probs * probs.log(), dim=-1).item()

            # Compute dynamic temperature
            temperature = T0 * (N ** (1 / (theta * entropy + 1e-8)))  # Avoid division by zero

            # Sample from adjusted distribution
            scaled_logits = logits / temperature
            scaled_probs = F.softmax(scaled_logits, dim=-1)
            next_token = torch.multinomial(scaled_probs, num_samples=1)

            # Append new token
            generated_ids = torch.cat([generated_ids, next_token], dim=-1)

            # Stop if EOS is reached
            if next_token.item() == tokenizer.eos_token_id:
                break

    answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    if f'\\boxed{{{df.iloc[i]["answer"]}}}' in answer:
        true_answer += 1

    print(f'\nSample: {i} --- Answer: {answer[-100:]}')
    print(f'Sample: {i} --- Number correct answer: {true_answer}')
